In [1]:
# from os import chdir
# from google.colab import drive

# drive.mount("/content/drive", force_remount=True)
# chdir("/content/drive/MyDrive/Eliott/files/")

# !pip install tweet-preprocessor

ModuleNotFoundError: No module named 'google.colab'

In [1]:
# IMPORTS

import preprocessing as pp
import hyperparameters as hp
import training_testing as tt
import json
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier



%load_ext autoreload
%autoreload 2


%pip install ipython-autotime
%load_ext autotime


2022-04-07 22:39:01.262404: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-07 22:39:01.262427: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Note: you may need to restart the kernel to use updated packages.
time: 5.07 ms (started: 2022-04-07 22:39:04 -04:00)


In [5]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/eliott/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

time: 4.93 s (started: 2022-04-07 22:40:18 -04:00)


In [6]:
file_name_train = 'Corona_NLP_train.csv'
file_name_test = 'Corona_NLP_test.csv'
X_train, y_train = pp.prepare_dataframe(file_name_train,lemmatising='nltk', original_class=False)
X_test, y_test = pp.prepare_dataframe(file_name_test,lemmatising='nltk', original_class=False)

100%|██████████| 3798/3798 [00:00<00:00, 16037.55it/s]

time: 8.42 s (started: 2022-04-07 22:40:26 -04:00)


On enlève : 
- Les URLS
- Hashtags
- Mentions
- Mots réservés
- Emojis et smileys

- 35525 mots sans lemmatisation
- 30794 avec lemmatisation

In [13]:
pipeline_sgd = hp.pipeline_sgd
pipeline_gb = hp.pipeline_gb
pipeline_rf = hp.pipeline_rf
pipeline_lr = hp.pipeline_lr
pipeline_per = hp.pipeline_per
pipeline_svc = hp.pipeline_svc

time: 1.11 ms (started: 2022-04-07 22:41:41 -04:00)


# Recherche d'Hyperparamètres

### Saving Hyperparameters in JSON

We will first load old weights and updates only if needed. Then we can use the model with best params in the end.

In [18]:
with open('data.json') as json_file:
    dico = json.load(json_file)

### SGD

In [35]:
%%time
grid_sgd = hp.grid_Search("SGD", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


/home/eliott/envs/RNenv/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Score:  0.8664868049029841
Best Params:  {'clf__alpha': 1.2000000000000002e-05, 'clf__penalty': 'l1', 'tfidf__max_df': 0.5, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1)}
CPU times: user 1min 44s, sys: 8.99 s, total: 1min 53s
Wall time: 30min 8s
time: 30min 8s (started: 2022-04-07 22:46:41 -04:00)


In [38]:
dico["SGD"]["best_score"] = 0.8763273700753711

In [39]:
hp.if_Save("SGD", dico, grid_sgd)

#### New Best Score for  SGD  :  0.8769590395125022


### Gradient boosting

In [10]:
%%time
grid_gb = hp.grid_Search("GB", X_train=X_train, y_train=y_train, subset=1000) 

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Score:  0.6
Best Params:  {'clf__learning_rate': 0.8, 'clf__n_estimators': 1600}
CPU times: user 14.6 s, sys: 208 ms, total: 14.8 s
Wall time: 58.5 s


In [ ]:
hp.if_Save("GB", dico, grid_gb)

### Random Forest

In [ ]:
%%time
grid_rf = hp.grid_Search("RF", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best Score:  0.7047644196092446
Best Params:  {'clf__criterion': 'entropy', 'clf__max_depth': None, 'clf__max_features': 'log2', 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 32, 'clf__n_estimators': 200}
CPU times: user 2min 29s, sys: 14 s, total: 2min 43s
Wall time: 2h 30min 23s


In [ ]:
hp.if_Save('RF', dico, grid_rf)

### Logistic Regression

In [ ]:
%%time
grid_lr = hp.grid_Search("LR", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Score:  0.8230677619122002
Best Params:  {'clf__C': 10, 'clf__tol': 1e-12}
CPU times: user 1min 20s, sys: 2min 15s, total: 3min 35s
Wall time: 36min 42s


In [ ]:
hp.if_Save('LR', dico, grid_lr)

### Perceptron

In [ ]:
%%time
grid_per = hp.grid_Search("PER", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Best Score:  0.818353925564533
Best Params:  {'clf__alpha': 3.04e-06, 'clf__penalty': 'l1'}
CPU times: user 1min 10s, sys: 7.51 s, total: 1min 17s
Wall time: 16min 16s


In [ ]:
hp.if_Save('PER', dico, grid_per)

#### Observation 

La regression logistique n'est qu'un perceptron avec une sigmoid en fonction d'activation.
On voit que la Regression Logistique a de meilleures performances à l'issue de la recherche d'hyperparamètres mais pas de loin. Par ailleurs le temps d'entrainement est considérablement plus élevé pour la regression logistique (du au calcul de l'exponentiel). Nous verrons par la suite quel modèle il est préférable de conserver. 

### LinearSVC

In [ ]:
%%time
grid_svc = hp.grid_Search("SVC", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_classes.py", line 272, in fit
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sk

Best Score:  0.868916566574562
Best Params:  {'clf__dual': False, 'clf__loss': 'squared_hinge', 'clf__penalty': 'l1'}
CPU times: user 11.5 s, sys: 524 ms, total: 12.1 s
Wall time: 1min 7s


In [ ]:
hp.if_Save('SVC', dico, grid_svc)

### Charging the best parameters for training

In [14]:
# Opening JSON file
with open('data.json') as json_file:
    data = json.load(json_file)

data_sgd = data['SGD']
data_rf = data['RF']
data_gb = data['GB']
data_lr = data['LR']
data_per = data['Perceptron']
data_svc = data['SVC']

time: 1.31 ms (started: 2022-04-07 22:41:47 -04:00)


### Training on all training data and testing on test data

In [15]:
def testing(model):
    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)
    accuracy_train = accuracy_score(y_train,predictions_train )
    accuracy_test = accuracy_score(y_test,predictions_test )
    print(f"train_accuracy : {accuracy_train} \ntest_accuracy : {accuracy_test}  ")

time: 1.03 ms (started: 2022-04-07 22:41:49 -04:00)


### SGD

In [16]:
data_sgd

{'params': {'clf__alpha': 1.2000000000000002e-05,
  'clf__penalty': 'l1',
  'tfidf__max_df': 0.75,
  'tfidf__min_df': 1,
  'tfidf__ngram_range': [1, 1]},
 'best_score': 0.8769590395125022}

time: 2.74 ms (started: 2022-04-07 22:41:50 -04:00)


In [17]:
model_sgd_best = pipeline_sgd
model_sgd_best.set_params(**(data_sgd["params"]),clf__n_jobs=-1) 

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=[1, 1])),
                ('clf',
                 SGDClassifier(alpha=1.2000000000000002e-05, n_jobs=-1,
                               penalty='l1'))])

time: 5.32 ms (started: 2022-04-07 22:41:51 -04:00)


In [18]:
%%time
"""
Si on a fait la recherche avant, on prend le best_model du gridSearchCv,
Sinon, on reconstruit le modèle avec les best paramètres et on le fit à toutes les données d'entrainement. 
"""

model_sgd_best.fit(X_train,y_train)


CPU times: user 1.41 s, sys: 1.14 s, total: 2.56 s
Wall time: 844 ms


Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=[1, 1])),
                ('clf',
                 SGDClassifier(alpha=1.2000000000000002e-05, n_jobs=-1,
                               penalty='l1'))])

time: 849 ms (started: 2022-04-07 22:41:54 -04:00)


In [19]:
testing(model_sgd_best)

train_accuracy : 0.9285905192312365 
test_accuracy : 0.8723012111637704  
time: 556 ms (started: 2022-04-07 22:41:58 -04:00)


### Random Forest

In [20]:
model_rf_best = pipeline_rf
model_rf_best.set_params(**(data_rf["params"]),clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

time: 5.3 ms (started: 2022-04-07 22:42:56 -04:00)


In [21]:
%%time

model_rf_best.fit(X_train,y_train)

CPU times: user 20.9 s, sys: 127 ms, total: 21.1 s
Wall time: 2.07 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

time: 2.08 s (started: 2022-04-07 22:42:59 -04:00)


In [22]:
testing(model_rf_best)

train_accuracy : 0.8644701994800399 
test_accuracy : 0.6824644549763034  
time: 826 ms (started: 2022-04-07 22:43:10 -04:00)


### Gradient Boosting 

In [ ]:
model_gb_best = pipeline_gb
model_gb_best.set_params(**(data_gb["params"]))

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.4,
                                            n_estimators=1600))])

In [ ]:
%%time
model_gb_best.fit(X_train,y_train)

CPU times: user 29min 46s, sys: 4.29 s, total: 29min 51s
Wall time: 29min 43s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.4,
                                            n_estimators=1600))])

In [ ]:
testing(model_gb_best)

train_accuracy : 0.9979104405082975 
test_accuracy : 0.8457082675092154  


### Logistic Regression

In [23]:
model_lr_best = pipeline_lr
model_lr_best.set_params(**(data_lr['params']),clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

time: 4.34 ms (started: 2022-04-07 22:43:36 -04:00)


In [24]:
%%time

model_lr_best.fit(X_train,y_train)

CPU times: user 588 ms, sys: 354 ms, total: 942 ms
Wall time: 12.5 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

time: 12.5 s (started: 2022-04-07 22:43:39 -04:00)


In [25]:
testing(model_lr_best)

train_accuracy : 0.9722525937264621 
test_accuracy : 0.8270142180094787  
time: 546 ms (started: 2022-04-07 22:44:10 -04:00)


### Perceptron

In [26]:
model_per_best = pipeline_per
model_per_best.set_params(**(data_per['params']),clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

time: 3.7 ms (started: 2022-04-07 22:44:41 -04:00)


In [27]:
%%time
model_per_best.fit(X_train,y_train)

CPU times: user 1.17 s, sys: 1.12 s, total: 2.29 s
Wall time: 772 ms


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

time: 779 ms (started: 2022-04-07 22:44:42 -04:00)


In [28]:
testing(model_per_best)

train_accuracy : 0.8986563646524285 
test_accuracy : 0.809373354397051  
time: 550 ms (started: 2022-04-07 22:44:45 -04:00)


### SVC

In [29]:
model_svc_best = pipeline_svc
model_svc_best.set_params(**(data_svc['params']))

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

time: 3.25 ms (started: 2022-04-07 22:44:49 -04:00)


In [30]:
%%time

model_svc_best.fit(X_train,y_train)

CPU times: user 4.75 s, sys: 10.5 ms, total: 4.76 s
Wall time: 4.77 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

time: 4.77 s (started: 2022-04-07 22:44:50 -04:00)


In [31]:
testing(model_svc_best)

train_accuracy : 0.9423913307578298 
test_accuracy : 0.8604528699315429  
time: 561 ms (started: 2022-04-07 22:44:57 -04:00)


### Voting Classifier

In [34]:
clf1 = model_sgd_best

clf2 = model_rf_best

clf3 = model_gb_best

clf4 = model_lr_best

clf5 = model_per_best

clf6 = model_svc_best


eclf1 = VotingClassifier(
     estimators=[('sgd', clf1), ('rf', clf2), ('gb', clf3), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')

eclf2 = VotingClassifier(
     estimators=[('sgd', clf1), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')


NameError: name 'model_gb_best' is not defined

time: 14.4 ms (started: 2022-04-07 22:45:31 -04:00)


In [ ]:
%%time

eclf1.fit(X_train,y_train)

CPU times: user 30min 13s, sys: 3.55 s, total: 30min 17s
Wall time: 30min 37s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               SGDClassifier(alpha=1.28206e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('rf',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               RandomForestClassifier(criterion='entropy',
                                                                      max_features='log2',
                                                                      min_samples_split=32,
                                                                      n_estimators=200,
                                                                      n_j

In [ ]:
testing(eclf1)

train_accuracy : 0.9785698666083534 
test_accuracy : 0.8701948393891522  


In [32]:
%%time

eclf2.fit(X_train,y_train)

NameError: name 'eclf2' is not defined

time: 113 ms (started: 2022-04-07 22:45:17 -04:00)


In [ ]:
testing(eclf2)

train_accuracy : 0.9609057997424496 
test_accuracy : 0.8654555028962612  


### Reccurent Network

In [11]:
rnn = tt.RNN(X_train, y_train, y_test)

##### CREATING RNN #####
Vocabulary size: 38129

Example:

Sentence:
Cashier at grocery store was sharing his insights on To prove his credibility he commented "I'm in Civics class so I know what I'm talking about".

After tokenizing :
[1199, 15, 25, 18, 68, 1287, 242, 811, 10, 2, 2910, 242, 9777, 184, 9778, 220, 6, 18319, 1556, 53, 12, 124, 60, 220, 936, 58]

After padding :
[ 1199    15    25    18    68  1287   242   811    10     2  2910   242
  9777   184  9778   220     6 18319  1556    53    12   124    60   220
   936    58     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 63, 16)            610064    
                  

In [12]:
rnn.train() # Train the model

##### TRAINING #####
Epoch 1/5
1158/1158 [==============================] - 87s 74ms/step - loss: 0.1342 - accuracy: 0.7230 - val_loss: 0.0815 - val_accuracy: 0.8523
Epoch 2/5
1158/1158 [==============================] - 88s 76ms/step - loss: 0.0669 - accuracy: 0.8984 - val_loss: 0.0612 - val_accuracy: 0.8892
Epoch 3/5
1158/1158 [==============================] - 84s 72ms/step - loss: 0.0472 - accuracy: 0.9351 - val_loss: 0.0576 - val_accuracy: 0.8885
Epoch 4/5
1158/1158 [==============================] - 79s 68ms/step - loss: 0.0370 - accuracy: 0.9544 - val_loss: 0.0584 - val_accuracy: 0.8841
Epoch 5/5
1158/1158 [==============================] - 80s 69ms/step - loss: 0.0315 - accuracy: 0.9640 - val_loss: 0.0584 - val_accuracy: 0.8892
##### TRAINING COMPLETED #####


In [16]:
rnn.print_history() 

##### PRINTING HISTORY #####


##### PRINTING HISTORY COMPLETED #####


In [15]:
rnn.test(X_test) # Test the model

##### TESTING #####
119/119 [==============================] - 2s 20ms/step - loss: 0.0614 - accuracy: 0.8776
Test loss: 0.06137406453490257
Test Accuracy: 0.8775671124458313
##### TESTING COMPLETED #####


(0.06137406453490257, 0.8775671124458313)

### Tests perso

In [ ]:
tests_perso = ["I think covid is the most horrible threat we ever faced",
                "I love covid, thanks to it I can see my family much more often and I don't have to comute as much",
                "I would love to come to your birthday party, but I got covid, I have to stay confined"
]

In [ ]:
predictions_out = eclf2.predict(tests_perso)


In [ ]:
predictions_out

array([-1,  1,  1])